# MNIST 데이터셋 : 사람 손글씨 데이터
https://ko.wikipedia.org/wiki/MNIST_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B2%A0%EC%9D%B4%EC%8A%A4

In [1]:
"""
전체적인 구조

# 첫번째 레이어
합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화함수(Activation function) ReLU
맥스풀링(kernel_size-2, stride=2)

# 두번째 레이어
# 첫번쨰 레이어에서 out과 두번쨰 in 숫자 맞춰주기
합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화함수(Activation function) ReLU
맥스풀링(kernel_size-2, stride=2)

# 세번째 레이어
batch_size x 7 x 7 x 64 => batch size = 3136(특성맵을 펼치는 레이어)
Fully cinnected(전결합층 뉴런 10개) + 활성화 함수 softmax
"""

'\n전체적인 구조\n\n# 첫번째 레이어\n합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화함수(Activation function) ReLU\n맥스풀링(kernel_size-2, stride=2)\n\n# 두번째 레이어\n# 첫번쨰 레이어에서 out과 두번쨰 in 숫자 맞춰주기\n합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화함수(Activation function) ReLU\n맥스풀링(kernel_size-2, stride=2)\n\n# 세번째 레이어\nbatch_size x 7 x 7 x 64 => batch size = 3136(특성맵을 펼치는 레이어)\nFully cinnected(전결합층 뉴런 10개) + 활성화 함수 softmax\n'

### 모델을 만드는 과정

In [2]:
# 구현에 필요한 라이브러리 불러오기

import torch
import torch.nn as nn

# torch.nn.Conv2d() = nn.Conv2d()
# as를 선언하지 않았을 경우 torch.nn.Conv2d()
# as를 선언한 경우 nn.Conv2d()

In [3]:
# 임의의 텐서 생성 (1,1,28,28) -> MNIST 데이터를 받아오기 위한 빈 공간

inputs = torch.Tensor(1,1,28,28)
print("텐서의 크기: {}".format(inputs.shape))

텐서의 크기: torch.Size([1, 1, 28, 28])


In [4]:
# 첫번째 합성곱층 선언하기
conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1) # Conv2d내의 인자값(input_channel, ouput_channel, 커널 사이즈, 패딩)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
# 두번째 합성곱층 선언하기
# 두번째에서는 인자값만 선언해도 됨
conv2 = nn.Conv2d(32, 64, 3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [6]:
# 세번째 합성곱층 선언하기(풀링 레이어)
# 두번째 레이어까지의 특성맵을 펼치는 과정
pool = nn.MaxPool2d(2) # 간단히 선언하면 선언된 하나의 정수값을 통해서 kernel_size, stride가 결정되게 된다.
print(pool)

# pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False) # 위와 동일

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [7]:
# 선언된 레이어를 연결하는 과정
# 첫번째 레이어 지나가기
out = conv1(inputs) # 임의의 값이 conv레이어를 지나게 된 out
print(out.shape) # output_channel 이 32로 변한 것을 확인 # inputs.shape

torch.Size([1, 32, 28, 28])


In [8]:
# 컨벌루셔널 레이어를 거쳐 갈 경우 마지막에 pooling layer가 존재함
out = pool(out) # conv레이어에서 나올때 맥스풀링하게 되는 과정
print(out.shape)

torch.Size([1, 32, 14, 14])


In [9]:
# 두번째 레이어 지나가기
out = conv2(out) # 임의의 값이 conv레이어를 지나게 된 out
print(out.shape)

torch.Size([1, 64, 14, 14])


In [10]:
# 두번째 컨벌루셔널 레이어를 거쳐 갈 경우 마지막에 pooling layer가 존재함
out = pool(out)
print(out.shape) 

torch.Size([1, 64, 7, 7])


In [11]:
# view() 함수를 사용해서 텐서를 펼치는 작업 (탠서를 통합한다) : 전체 차원이 하나로 합쳐진 batch_size=3136
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [12]:
# fully connected : 전체를 연결(기초적인 신경망)
# 통합된 3136크기의 텐서를 통해 0~9까지의 결과를 갖게 되기 때문에 10개의 output
fc = nn.Linear(3136, 10)# 3136개를 input으로 받고 10개의 클래스를 output으로 
out = fc(out)
print(out.shape)

# 위까지가 전체 모델 생성 과정

torch.Size([1, 10])


### MNIST데이터셋을 불러와서 classification 진행

In [13]:
# 위에서 선언한 신경망을 통해서 MNIST(숫자 데이터셋) 분류 문제 해결하기
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init # 초기화 라이브러리

In [14]:
# GPU 가속화를 위한 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.is_available()함수는 False/True의 boolean형태로 나타나게 되는데, 
# CUDA를 사용 가능할 경우 True를 반환하기 때문에 device='cuda'가 되고, 
# 사용이 불가능할 경우 False를 반환하기 때문에 else 조건을 만족하는 device='cpu'가 됨

# GPU사용시에 고정시킬 시드를 선언
torch.manual_seed(777)

# 만약 CPU가 아닌 GPU를 사용해서 훈련을 진행할 경우 고정된 시드를 통해 GPU로 학습
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [15]:
print(torch.cuda.is_available()) # gpu 사용 가능하다면 True # False이면 cpu를 사용한다(위에서 조건 설정)

True


In [16]:
# 학습에 사용되는 변수(파라미터 : 학습률, 에폭, 배치사이즈 등) 선언
learning_rate = 0.001
training_epoch = 15
batch_size = 100

In [17]:
# MNIST train, test 데이터셋을 torchvision.datasets를 통해 비교적으로 쉽게 가져오기

mnist_train = dsets.MNIST(root='MNIST_data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

In [18]:
# 데이터로더를 이용해서 배치크기를 지정
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

In [19]:
class CNN(torch.nn.Module):# CNN 구조를 사용한 모델을 선언하기 위한 클래스
    def __init__(self): # 초기화 함수, 동시에 모델의 구조를 선언하는 함수
        super(CNN, self).__init__()
        
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(),
                                         torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(),
                                         torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x): # 레이어를 거치며 계산되는 가중치를 전달하기 위해 사용하는 함수
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        
        return out

In [20]:
# CNN 클래스 호출
# cuda든 cpu를 사용하든 연산을 하기위해 어떤 device에 부여할 것인지 설정하는 것
model = CNN().to(device) 
# model이라는 객체를 통해서 CNN 클래스가 선언이 되고, 이는, to(device)를 통해 gpu에서 연산을 하게 됨 

In [21]:
# 비용함수(cost function = loss function)는 크로스엔트로피로 선언
# 옵티마이저는 Adam으로 설정
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
# 총 batch 수 
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch)) # 총 6만개의 데이터를 가지고 600번을 반복하기에

# MNIST dataset은 60,000개의 데이터를 가지고 있는데, 
#batch_size를 100번으로 설정했으므로 총 600의 batch_size를 가지게 됨

총 배치의 수 : 600


In [23]:
# 학습하기
for epoch in range(training_epoch):
    avg_cost = 0 # cost의 총 평균을 초기화
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    # 뒤에서부터 몇 글자를 출력할지 # 소수점의 경우에는 .(comma)자리수
    print('[Epoch : {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost)) 

[Epoch :    1] cost = 0.225674048
[Epoch :    2] cost = 0.0631026775
[Epoch :    3] cost = 0.0462856963
[Epoch :    4] cost = 0.0374607854
[Epoch :    5] cost = 0.0315512568
[Epoch :    6] cost = 0.0261680596
[Epoch :    7] cost = 0.021803556
[Epoch :    8] cost = 0.0184477028
[Epoch :    9] cost = 0.0164121781
[Epoch :   10] cost = 0.0135204783
[Epoch :   11] cost = 0.0101817129
[Epoch :   12] cost = 0.00981508102
[Epoch :   13] cost = 0.00831910875
[Epoch :   14] cost = 0.00708321994
[Epoch :   15] cost = 0.00677147135


In [24]:
# 학습한 결과를 테스트하는 과정
with torch.no_grad(): # 더 이상 학습을 진행시키지 않기 위해 no_grad()사용
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    
    print("Accuracy:", accuracy.item())

Accuracy: 0.9870999455451965


C:\Users\10\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\10\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
